In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from tqdm import tqdm
import numpy as np
import os
import cv2
from scipy.spatial.distance import directed_hausdorff

In [4]:
train_img = []
train_labels = []

test_img = []
test_labels = []

path_train = '/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Training/'
path_test = '/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Testing/'

In [5]:
def process_images(path):
    images = []
    labels = []
    for category in os.listdir(path):
        category_path = os.path.join(path, category)
        if os.path.isdir(category_path):  # Ensure it's a directory
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    resized_img = cv2.resize(img, (img_size, img_size))
                    if len(resized_img.shape) == 2:  # Convert grayscale to RGB
                        resized_img = cv2.cvtColor(resized_img, cv2.COLOR_GRAY2RGB)
                    images.append(resized_img)
                    labels.append(category)
                else:
                    print(f"Warning: Skipping file {img_path} as it is not a valid image.")
    return images, labels

In [6]:
transform = transforms.Compose([
    transforms.ToPILImage(),   # Convert from NumPy array to PIL Image
    transforms.Resize((300, 300)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),     # Convert from PIL Image to PyTorch Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1)
        self.batch_norm = nn.BatchNorm3d(out_channels)
    
    def forward(self, x):
        x = F.relu(self.batch_norm(self.conv1(x)))
        x = F.relu(self.batch_norm(self.conv2(x)))
        return x

class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.conv_block = ConvBlock(in_channels, out_channels)
        self.pool = nn.Conv3d(out_channels, out_channels, kernel_size=2, stride=2)
    
    def forward(self, x):
        x = self.conv_block(x)
        pooled = self.pool(x)
        return x, pooled

class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpBlock, self).__init__()
        self.conv_block = ConvBlock(in_channels, out_channels)
        self.up = nn.ConvTranspose3d(in_channels, out_channels, kernel_size=2, stride=2)
    
    def forward(self, x, bridge):
        upsampled = self.up(x)
        out = torch.cat([upsampled, bridge], dim=1)
        out = self.conv_block(out)
        return out

class VNet(nn.Module):
    def __init__(self, num_classes):
        super(VNet, self).__init__()
        self.down1 = DownBlock(1, 16)
        self.down2 = DownBlock(16, 32)
        self.down3 = DownBlock(32, 64)
        self.down4 = DownBlock(64, 128)
        
        self.bottleneck = ConvBlock(128, 256)
        
        self.up1 = UpBlock(256, 128)
        self.up2 = UpBlock(128, 64)
        self.up3 = UpBlock(64, 32)
        self.up4 = UpBlock(32, 16)
        
        self.final_conv = nn.Conv3d(16, num_classes, kernel_size=1)
    
    def forward(self, x):
        bridge1, down1 = self.down1(x)
        bridge2, down2 = self.down2(down1)
        bridge3, down3 = self.down3(down2)
        bridge4, down4 = self.down4(down3)
        
        bottleneck = self.bottleneck(down4)
        
        up1 = self.up1(bottleneck, bridge4)
        up2 = self.up2(up1, bridge3)
        up3 = self.up3(up2, bridge2)
        up4 = self.up4(up3, bridge1)
        
        out = self.final_conv(up4)
        return out

In [ ]:
label_map = {'no_tumor': 0, 'glioma_tumor': 1, 'meningioma_tumor': 2, 'pituitary_tumor': 3}
train_labels_encoded = [label_map[label] for label in train_labels]
test_labels_encoded = [label_map[label] for label in test_labels]

In [ ]:
train_dataset = BrainTumorDataset(train_img, train_labels_encoded, transform=transform)
test_dataset = BrainTumorDataset(test_img, test_labels_encoded, transform=transform)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder(root=path_train, transform=transform)
test_dataset = ImageFolder(root=path_test, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Initialize model, loss function, and optimizer
model = VNet(in_channels=3, out_channels=4)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Define Dice Coefficient
def dice_coefficient(y_true, y_pred):
    smooth = 1e-6
    y_true_f = y_true.contiguous().view(-1)
    y_pred_f = y_pred.contiguous().view(-1)
    intersection = (y_true_f * y_pred_f).sum()
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)

# Define Hausdorff Distance
def hausdorff_distance(y_true, y_pred):
    y_true = y_true.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    return max(directed_hausdorff(y_true, y_pred)[0], directed_hausdorff(y_pred, y_true)[0])

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images = images.to('cuda' if torch.cuda.is_available() else 'cpu')
        labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from tqdm import tqdm
import numpy as np
import os
import cv2
from scipy.spatial.distance import directed_hausdorff

# Dataset Class
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from tqdm import tqdm
import numpy as np
import os
import cv2
from scipy.spatial.distance import directed_hausdorff

# Dataset Class
class BrainTumorDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # Ensure image has 3 channels
        if len(image.shape) == 2:  # Grayscale image
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        elif image.shape[2] == 4:  # Image with alpha channel
            image = image[:, :, :3]  # Remove alpha channel

        if self.transform:
            image = self.transform(image)

        return image, label


# Data Augmentation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((300, 300)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# V-Net Model
class VNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=4):
        super(VNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(in_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True),
            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(64, 32, kernel_size=2, stride=2),
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True),
            nn.Conv3d(32, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Image Processing Function
def process_images(path, img_size):
    images = []
    labels = []
    for category in os.listdir(path):
        category_path = os.path.join(path, category)
        if os.path.isdir(category_path):  # Ensure it's a directory
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    resized_img = cv2.resize(img, (img_size, img_size))
                    if len(resized_img.shape) == 2:  # Convert grayscale to RGB
                        resized_img = cv2.cvtColor(resized_img, cv2.COLOR_GRAY2RGB)
                    elif resized_img.shape[2] == 4:  # Handle images with an alpha channel
                        resized_img = resized_img[:, :, :3]
                    images.append(resized_img)
                    labels.append(category)
                else:
                    print(f"Warning: Skipping file {img_path} as it is not a valid image.")
    return images, labels

# Define paths
path_train = '/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Training/'
path_test = '/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Testing/'
img_size = 300

# Process training and testing data
train_img, train_labels = process_images(path_train, img_size)
test_img, test_labels = process_images(path_test, img_size)

train_img = np.array(train_img)
test_img = np.array(test_img)

# Encode labels
label_map = {'no_tumor': 0, 'glioma_tumor': 1, 'meningioma_tumor': 2, 'pituitary_tumor': 3}
train_labels_encoded = [label_map[label] for label in train_labels]
test_labels_encoded = [label_map[label] for label in test_labels]

# Create dataset and dataloaders
train_dataset = BrainTumorDataset(train_img, train_labels_encoded, transform=transform)
test_dataset = BrainTumorDataset(test_img, test_labels_encoded, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Verify the shape of a batch
for images, labels in train_loader:
    print(f'Batch shape: {images.shape}')
    break

# Initialize model, loss function, and optimizer
model = VNet(in_channels=3, out_channels=4)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Define Dice Coefficient
def dice_coefficient(y_true, y_pred):
    smooth = 1e-6
    y_true_f = y_true.contiguous().view(-1)
    y_pred_f = y_pred.contiguous().view(-1)
    intersection = (y_true_f * y_pred_f).sum()
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)

# Define Hausdorff Distance
def hausdorff_distance(y_true, y_pred):
    y_true = y_true.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    return max(directed_hausdorff(y_true, y_pred)[0], directed_hausdorff(y_pred, y_true)[0])

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images = images.to('cuda' if torch.cuda.is_available() else 'cpu')
        labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1)
        self.batch_norm = nn.BatchNorm3d(out_channels)
    
    def forward(self, x):
        x = F.relu(self.batch_norm(self.conv1(x)))
        x = F.relu(self.batch_norm(self.conv2(x)))
        return x

class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.conv_block = ConvBlock(in_channels, out_channels)
        self.pool = nn.Conv3d(out_channels, out_channels, kernel_size=2, stride=2)
    
    def forward(self, x):
        x = self.conv_block(x)
        pooled = self.pool(x)
        return x, pooled

class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpBlock, self).__init__()
        self.conv_block = ConvBlock(in_channels, out_channels)
        self.up = nn.ConvTranspose3d(in_channels, out_channels, kernel_size=2, stride=2)
    
    def forward(self, x, bridge):
        upsampled = self.up(x)
        out = torch.cat([upsampled, bridge], dim=1)
        out = self.conv_block(out)
        return out

class VNet(nn.Module):
    def __init__(self, num_classes):
        super(VNet, self).__init__()
        self.down1 = DownBlock(1, 16)
        self.down2 = DownBlock(16, 32)
        self.down3 = DownBlock(32, 64)
        self.down4 = DownBlock(64, 128)
        
        self.bottleneck = ConvBlock(128, 256)
        
        self.up1 = UpBlock(256, 128)
        self.up2 = UpBlock(128, 64)
        self.up3 = UpBlock(64, 32)
        self.up4 = UpBlock(32, 16)
        
        self.final_conv = nn.Conv3d(16, num_classes, kernel_size=1)
    
    def forward(self, x):
        bridge1, down1 = self.down1(x)
        bridge2, down2 = self.down2(down1)
        bridge3, down3 = self.down3(down2)
        bridge4, down4 = self.down4(down3)
        
        bottleneck = self.bottleneck(down4)
        
        up1 = self.up1(bottleneck, bridge4)
        up2 = self.up2(up1, bridge3)
        up3 = self.up3(up2, bridge2)
        up4 = self.up4(up3, bridge1)
        
        out = self.final_conv(up4)
        return out

# Create a sample dataset
class BrainTumorDataset(Dataset):
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]
        return torch.tensor(image, dtype=torch.float32), torch.tensor(mask, dtype=torch.long)

# Load your data
# Note: Replace these with the actual paths to your training and testing datasets
train_images = np.load('train_images.npy')
train_masks = np.load('train_masks.npy')
test_images = np.load('test_images.npy')
test_masks = np.load('test_masks.npy')

# Create DataLoader
train_dataset = BrainTumorDataset(train_images, train_masks)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

test_dataset = BrainTumorDataset(test_images, test_masks)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# Define model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VNet(num_classes=4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 50
best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    total_train = 0
    train_dice = 0.0
    train_hausdorff = 0.0

    for images, masks in train_loader:
        images = images.unsqueeze(1).to(device)  # Add channel dimension
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == masks).sum().item()
        total_train += masks.numel()

        for pred, mask in zip(predicted, masks):
            train_dice += dice_coefficient(pred, mask)
            train_hausdorff += hausdorff_distance(pred, mask)

    train_loss /= len(train_loader.dataset)
    train_accuracy = train_correct / total_train
    train_dice /= len(train_loader.dataset)
    train_hausdorff /= len(train_loader.dataset)

    model.eval()
    test_correct = 0
    total_test = 0
    test_dice = 0.0
    test_hausdorff = 0.0

    with torch.no_grad():
        for images, masks in test_loader:
            images = images.unsqueeze(1).to(device)
            masks = masks.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_correct += (predicted == masks).sum().item()
            total_test += masks.numel()

            for pred, mask in zip(predicted, masks):
                test_dice += dice_coefficient(pred, mask)
                test_hausdorff += hausdorff_distance(pred, mask)

    test_accuracy = test_correct / total_test
    test_dice /= len(test_loader.dataset)
    test_hausdorff /= len(test_loader.dataset)

    print(f'Epoch: {epoch + 1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
          f'Train Dice: {train_dice:.4f}, Train Hausdorff: {train_hausdorff:.4f}, '
          f'Test Accuracy: {test_accuracy:.4f}, Test Dice: {test_dice:.4f}, Test Hausdorff: {test_hausdorff:.4f}')

    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'vnet_best_checkpoint.pth')
        best_accuracy = test_accuracy

print(f'Training complete. Best accuracy: {best_accuracy:.4f}')

FileNotFoundError: [Errno 2] No such file or directory: 'train_images.npy'